In [1]:
import pandas as pd
import numpy as np
import os, sys
import random
import h5py
import math
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats
import itertools
import json

In [2]:
testing = False
rseed = 0

# read in the working directories and iteration number
if len(sys.argv) == 4:
    # set locations for working files
    automation_dir = sys.argv[1]
    attpcroot_dir = sys.argv[2]
    
    iteration = int(sys.argv[3])
else:
    if testing:
        print("TESTING MODE")
        automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/'
        attpcroot_dir = '/mnt/analysis/e17023/Adam/ATTPCROOTv2/'
        iteration = 1
    else:
        print("Usage: python tuning-params.py <automation_dir> <attpcroot_dir> <iteration>")
        raise ValueError("Incorrect number of arguments passed to tuning-params.py")
rseed += iteration # to ensure that the random seed is different for each iteration

TESTING MODE


In [3]:
def indicator_file(file_type, indicator_directory=automation_dir):
    df = pd.DataFrame([0])
    df.to_csv(indicator_directory + file_type + '.csv', index=False)
    print(file_type + ' FILE CREATED')

In [4]:
def energy_to_momentum(energy, particle):
    # input energy in KeV, convert to MeV
    energy = energy/1000

    # Mass values from NIST
    if particle == 'a':
        mass = 3727.3794066 # MeV/c^2
    elif particle == 'p':
        mass = 938.27208816 # MeV/c^2
    else:
        indicator_file('STOP')
        raise Exception('Error: particle must be "a" or "p"')
    momentum = np.sqrt(2*mass*energy)/1000 # GeV/c
    return momentum

In [5]:
def Analyze_H5(automation_dir, file, threshold, ADD_NOISE=False):
    def pca_func(data, n_components):
        
        # original method
        #pca = PCA(n_components=n_components)
        #pca.fit(data)
        #out = pca.transform(data)
        
        # fishtank-compatible method
        # Subtract the mean from the data to center it
        mean = np.mean(data, axis=0)
        data_centered = data - mean
        
        # Compute the covariance matrix
        cov = np.cov(data_centered, rowvar=False)

        # Compute the eigenvectors and eigenvalues of the covariance matrix
        eigvals, eigvecs = np.linalg.eig(cov)

        # Transform the data into the new coordinate system defined by the eigenvectors
        out = np.dot(data_centered, eigvecs)

        return out
    
    def dbscan(X, eps, min_samples):
        # Initialize variables
        labels = np.zeros(X.shape[0])
        cluster = 0

        # Compute distances between points
        dists = np.sqrt(((X[:, np.newaxis] - X) ** 2).sum(axis=2))

        # Iterate over each point
        for i in range(X.shape[0]):
            # If point already visited, continue
            if labels[i] != 0:
                continue

            # Find neighboring points
            neighbors = np.where(dists[i] <= eps)[0]

            # If not enough neighboring points, label as noise
            if len(neighbors) < min_samples:
                labels[i] = -1
                continue

            # Expand cluster
            cluster += 1
            labels[i] = cluster

            while len(neighbors) > 0:
                j = neighbors[0]
                if labels[j] == -1:
                    labels[j] = cluster
                elif labels[j] == 0:
                    labels[j] = cluster
                    new_neighbors = np.where(dists[j] <= eps)[0]
                    if len(new_neighbors) >= min_samples:
                        neighbors = np.concatenate((neighbors, new_neighbors))
                neighbors = neighbors[1:]

        return labels
    
    def remove_outliers(xset, yset, zset, eset, threshold):
        #Uses DBSCAN to find and remove outliers in 3D data
        # NEEDS ALTERNATE METHOD FOR FISHTANK COMPATIBILITY
        data = np.array([xset.T, yset.T, zset.T]).T

        # STANDARD METHOD
        #DBSCAN_cluster = DBSCAN(eps=7, min_samples=10).fit(data)
        #out_of_cluster_index = np.where(DBSCAN_cluster.labels_==-1)

        # FISHTANK METHOD
        labels = dbscan(data, eps=7, min_samples=10)
        out_of_cluster_index = np.where(labels==-1)

        del data
        rev = out_of_cluster_index[0][::-1]

        for i in rev:
            xset = np.delete(xset, i)
            yset = np.delete(yset, i)
            zset = np.delete(zset, i)
            eset = np.delete(eset, i)
        if len(xset) <= threshold:
            veto = True
        else:
            veto = False

        # testing
        veto = False
        
        return xset, yset, zset, eset, veto
    
    def track_len(xset, yset, zset):
        """
        Uses PCA to find the length of a track
        """
        veto_on_length = False
    
        # Form data matrix
        data = np.concatenate((xset[:, np.newaxis], 
                               yset[:, np.newaxis], 
                               zset[:, np.newaxis]), 
                               axis=1)
    
        # Use PCA to find track length
        principalComponents = pca_func(data, 3)
    
        principalDf = pd.DataFrame(data = principalComponents
                 , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
        
        track_len = 2.35*principalDf.std()[0]
        track_width = 2.35*principalDf.std()[1]
        track_depth = 2.35*principalDf.std()[2]
        #if track_len > 70:
        #    veto_on_length = True
        
        return track_len, veto_on_length, track_width, track_depth
    
    def main(h5file, threshold, ADD_NOISE):
        """
        This functions does the following: 
        - Converts h5 files into ndarrays. 
        - Removes outliers.
        - Calls PCA to return track length.
        - Sums mesh signal to return energy.
        """
        # Converts h5 files into ndarrays, and output each event dataset as a separte list
        num_events = int(len(list(h5file.keys()))/2) 
        
        good_events = []
        
        length_list = []
        width_list = []
        depth_list = []
        
        tot_energy = []
        tracemax_list = []
        peakw_list = []
        
        maxe_list = []
        stde_list = []
        padnum_list = []
        
        skipped_events = 0
        veto_events = 0
        
        #pbar = tqdm(total=num_events+1)
        for i in range(0, num_events):
            str_event = f"Event_[{i}]"
            
            # Apply pad threshold
            event = np.array(h5file[str_event][:])
            if len(event) <= threshold:
                skipped_events += 1
                #pbar.update(n=1)
                continue
                
            # Make copy of datasets
            dset_0_copyx = event['x']
            dset_0_copyy = event['y'] 
            dset_0_copyz = event['z'] - min(event['z'])
            dset_0_copye = event['A']
            
            # Apply veto condition
            R = 36                           # Radius of the pad plane
            r = np.sqrt(dset_0_copyx**2 + dset_0_copyy**2)
            statements = np.greater(r, R)    # Check if any point lies outside of R
        
            if np.any(statements) == True:
                veto_events += 1
                #pbar.update(n=1)
                continue
            
            
            # Call remove_outliers to get dataset w/ outliers removed
            dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, veto = remove_outliers(dset_0_copyx, dset_0_copyy, dset_0_copyz, dset_0_copye, threshold)
            veto = False
            if veto == True:
                skipped_events += 1
                #pbar.update(n=1)
                continue

            
            # Call track_len() to create lists of all track lengths
            length, veto_on_length, width, depth = track_len(dset_0_copyx, dset_0_copyy, dset_0_copyz)
            if veto_on_length == True:
                veto_events += 1
                #pbar.update(n=1)
                continue

            
            str_trace = f"Trace_[{i}]"
            trace = np.array(h5file[str_trace][:])
            max_val = np.argmax(trace)
            low_bound = max_val - 75
            if low_bound < 0:
                low_bound = 5
            upper_bound = max_val + 75
            if upper_bound > 511:
                upper_bound = 506
            trace = trace[low_bound:upper_bound]

            # STANDARD METHOD
            #baseObj=BaselineRemoval(trace)
            #trace=baseObj.IModPoly(polynomial_degree)

            # FISHTANK METHOD
            # determine the width of the peak in the trace and the location of the peak
            peakloc = np.argmax(trace)
            
            peakwidth1 = 0
            peakwidth2 = 0

            k = peakloc
            while trace[k] > np.min(trace) + np.std(trace):
                peakwidth1 += 1
                k += 1
            k = peakloc
            while trace[k] > np.min(trace) + np.std(trace):
                peakwidth2 += 1
                k -= 1

            # calculate the average of the trace outside of the peakwidth window on either side of the peak
            baseline = np.mean(np.concatenate((trace[:peakloc-peakwidth2], trace[peakloc+peakwidth1:])))
            # subtract the baseline from the trace
            trace = trace - baseline

            # ADD NOISE TO TRACE
            # noise of frequency ~0.25 with random phase
            if ADD_NOISE:
                noise_peak = np.sin(np.linspace(0, 2*np.pi*len(trace)*random.gauss(2.49815e-1, -8.27021e-3), len(trace)) + random.random()*2*np.pi)

                noise_background = np.zeros_like(trace)
                for freq in np.linspace(0.1, 0.5):
                    noise_background += np.sin(np.linspace(0, 2*np.pi*len(trace)*freq, len(trace)) + random.random()*2*np.pi) * random.gauss(1, 0.3)
                noise_background = noise_background - np.mean(noise_background)
                noise_background = noise_background / np.std(noise_background)


                noise = 13000*noise_peak + 2000 * noise_background

                # normalize noise to std of 1 and average of 0, then multiply by amplitude of real data (402.57 +- 106.40 std)
                noise = (noise - np.mean(noise))/np.std(noise) * random.gauss(402.57, 106.40)
                # add noise to trace
                trace = trace + noise
            
            # VETO ON TRACE SUM
            #if np.sum(trace) > 800000:
            #    veto_events += 1
            #    pbar.update(n=1)
            #    continue

            length_list.append(length)
            width_list.append(width)
            depth_list.append(depth)
            
            tot_energy.append(np.sum(trace))
            tracemax_list.append(np.max(trace))
            peakw_list.append(peakwidth1 + peakwidth2)
            
            maxe_list.append(np.max(dset_0_copye))
            stde_list.append(np.std(dset_0_copye))
            padnum_list.append(len(dset_0_copyx))

            # Track event number of good events
            good_events.append(i)  
            #pbar.update(n=1)
            
        # package lists into a dictionary
        output_dict = {
            "length": length_list,
            "width": width_list,
            "depth": depth_list,
            "trace_sum": tot_energy,
            "trace_max": tracemax_list,
            "peak_width": peakw_list,
            "max_e": maxe_list,
            "std_e": stde_list,
            "padnum": padnum_list}
        
        return output_dict
    
    h5f = h5py.File(automation_dir + 'simOutput/' + file, 'r')
    
    output_dict = main(h5file=h5f, threshold=5, ADD_NOISE=ADD_NOISE)
    
    results_df = pd.DataFrame(columns=['file', 'ptype', 'event#',
                                       'length', 'width', 'depth',
                                       'trace_sum', 'trace_max', 'peak_width',
                                       'max_e', 'std_e', 'padnum'])
    
    file_name = file.split('.h5')[0]
    ptype = file_name.split('-')[-1]
    
    for i in range(len(output_dict['length'])):
        results_df.loc[i] = [file_name, ptype, i,
                             output_dict['length'][i], output_dict['width'][i], output_dict['depth'][i],
                             output_dict['trace_sum'][i], output_dict['trace_max'][i], output_dict['peak_width'][i],
                             output_dict['max_e'][i], output_dict['std_e'][i], output_dict['padnum'][i]]
        
    return results_df

In [6]:
# first iteration setup
if iteration == 0:
    print('First iteration, setting up files')
    
    # create tuning log file
    tuning_log = pd.DataFrame(columns=['file', 'ptype', 'event#',
                                       'length', 'width', 'depth',
                                       'trace_sum', 'trace_max', 'peak_width',
                                       'max_e', 'std_e', 'padnum'])
    
    # READ IN REFERENCE FILES AND ANALYZE FOR COMPARISON
    reference_files = []
    # look for reference h5 files in simOutput
    for file in os.listdir(automation_dir + 'simOutput/'):
        if file.endswith('.h5') and file.startswith('ref-'):
            reference_files.append(file)
    
    # analyze reference files
    for file in reference_files:
        print('Analyzing reference file: ' + file)
        results = Analyze_H5(automation_dir, file, 1, ADD_NOISE=False)
        tuning_log = tuning_log.append(results, ignore_index=True)
    
    # write tuning log to csv
    tuning_log.to_csv(automation_dir + 'simOutput/tuning_log.csv', index=False)
    
    # SETUP OF FIRST ITERATION SIMULATION
    # initialize tuning parameters
    param_df = pd.DataFrame(columns=['Sim', 'Status', 'N', 'P0', 'E0', 'P1', 'E1', 'Xb', 'Yb', 'Zb1', 'Zb2', 'Threshold'])
    
    # prompt user for list of variables to tune, and initial values and variation
    tuning_params = {}
    print('Enter parameters to tune, along with initial value and percent bounding range (Separate by spaces)')
    print('Example: N 1000 10') # N variable, centered at 1000 with 10% bounds
    print('Input a blank line to finish')
    while True:
        user_input = input()
        if user_input == '':
            if len(tuning_params) == 0:
                print('No tuning parameters entered, please enter at least one')
            else:
                break
        else:
            split_input = user_input.split()
            if len(split_input) != 3:
                print('Invalid input, please try again')
            else:
                param, val, var = split_input
                if param in tuning_params or param in ['Sim', 'Status', 'P0', 'E0', 'P1', 'E1', 'Xb', 'Yb', 'Zb1', 'Zb2']:
                    print('Parameter specification invalid, please try again')
                else:
                    try :
                        val = float(val)
                        var = float(var)
                        tuning_params[param] = [float(val), val - (var * val / 100), val + (var * val / 100)] # store as [center, -bound, +bound]
                    except ValueError:
                        print('Invalid input, please try again')
    
    # write initial tuning parameters to param_df
    for ptype in tuning_log['ptype'].unique():
        # get p0, e0, p1, e1 from reference files
        ptypel = ptype.split(' ')
        p0, e0 = ptypel[0][-1], ptypel[0][:-1]
        print(p0, e0)
        if len(ptypel) == 2:
            p1, e1 = ptypel[1][-1], ptypel[1][:-1]
            print(p1, e1)
        else:
            p1, e1 = 'a', 0
        
        # name simulation(s)
        simname = 'tuning' + str(iteration) + '-' + ptype
        
        if len(param_df) == 0:
            param_df.loc[0] = [simname, 0, 100, p0, e0, p1, e1, 0, 0, 0, 0, 1]
        else:
            # copy previous row
            param_df.loc[len(param_df)] = param_df.loc[len(param_df)-1].copy()
        
        # write values to param_df
        param_df.loc[len(param_df)-1, 'Sim'] = simname
        param_df.loc[len(param_df)-1, 'P0'] = p0; param_df.loc[len(param_df)-1, 'E0'] = e0
        param_df.loc[len(param_df)-1, 'P1'] = p1; param_df.loc[len(param_df)-1, 'E1'] = e1
        
        # add columns to param_df for tuning parameters
        for param in tuning_params:
            # get initial value
            val = tuning_params[param][0]
            param_df.loc[len(param_df)-1, param] = val
            
    # add Score column to param_df
    param_df['Score'] = -1 # -1 indicates not yet scored
        
    # write param_df to csv
    param_df.to_csv(automation_dir + 'simInput/parameters.csv', index=False)
    
    # prompt user for DecayRate, Target, MaxIterations, BestN
    
    SavedSettings = {}
    print('Enter tuning parameters')
    SavedSettings['BaseVar'] = float(input('Base Variation: '))
    SavedSettings['DecayRate'] = float(input('DecayRate: '))
    SavedSettings['Target'] = float(input('Target: '))
    SavedSettings['MaxIterations'] = float(input('MaxIterations: '))
    SavedSettings['BestN'] = int(input('BestN: '))
    
    SavedSettings['ptypes'] = tuning_log['ptype'].unique().tolist()
    tuning_params['Settings'] = SavedSettings
        
    # save intial tuning parameters to json file
    with open(automation_dir + 'simOutput/tuning_params.json', 'w') as f:
        json.dump(tuning_params, f, indent=4)

In [7]:
def compare_analysis(tuning_log, refname, simname):
    # compare analysis of reference file to simulation file
    ref_data = tuning_log[tuning_log['file'] == refname]
    sim_data = tuning_log[tuning_log['file'] == simname]
    
    score = 0
    cols = 0
    
    sampled_cols = ['Length', 'Trace_sum'] # columns that are selected for, standard deviation should not be compared
    
    for col in ref_data.columns:
        if col in ['file', 'ptype', 'event#']:
            continue
        else:
            cols += 1
            #score += stats.ks_2samp(ref_data[col], sim_data[col])[0]
            score += np.abs(np.mean(ref_data[col]) - np.mean(sim_data[col])) / np.mean(ref_data[col]) # compare means
            #if col not in sampled_cols:
            #    score += np.abs(np.std(ref_data[col]) - np.std(sim_data[col])) / np.std(ref_data[col]) # compare stds
    
    return score

In [12]:
# subsequent iterations
# load in parameters.csv
param_df = pd.read_csv(automation_dir + 'simInput/parameters.csv')

# test if all queued simulations are complete
if iteration > 0 and len(param_df[param_df['Status'] == 0]) == 0:
    if param_df[param_df['Status'] == 1].shape[0] > 0:
        # temporarily rename unnamed output.h5 to corresponding simulation name
        for file in os.listdir(automation_dir + 'simOutput/'):
            if file == 'output.h5':
                new_name = param_df.loc[param_df['Status'] == 1, 'Sim'].values[0] + '.h5'
                os.rename(automation_dir + 'simOutput/output.h5', automation_dir + 'simOutput/' + new_name)        
    
    # analyze all simulations not yet analyzed
    tuning_log = pd.read_csv(automation_dir + 'simOutput/tuning_log.csv')
    for file in os.listdir(automation_dir + 'simOutput/'):
        if file.endswith('.h5') and file[:-3] not in tuning_log['file'].values:
            print('Analyzing file: ' + file)
            results = Analyze_H5(automation_dir, file, 1, ADD_NOISE=False)
            tuning_log = tuning_log.append(results, ignore_index=True)
    
    # calculate score for each simulation and write to param_df
    for i in range(len(param_df)):
        if param_df.loc[i, 'Score'] == -1: # if score has not been calculated
            refname = 'ref-' + str(param_df.loc[i, 'E0']) + param_df.loc[i, 'P0'] 
            if param_df.loc[i, 'P1'] != 'a' and param_df.loc[i, 'E1'] != 0:
                refname += ' ' + str(param_df.loc[i, 'E1']) + param_df.loc[i, 'P1']
            simname = param_df.loc[i, 'Sim']
            
            score = compare_analysis(tuning_log, refname, simname)
            param_df.loc[i, 'Score'] = score
    
     
    with open(automation_dir + 'simOutput/tuning_params.json', 'r') as f:
        tuning_params = json.load(f)
    
    # test for end of tuning
    if iteration >= tuning_params['Settings']['MaxIterations']:
        print('Tuning Iterations exceeded maximum, ending tuning')
        sys.exit()
    elif min(param_df['Score']) <= tuning_params['Settings']['Target']:
        print('Target score reached, ending tuning')
        sys.exit()
    
    best_param_df = 0
    
    # sort param_df by score for each particle type
    for p in tuning_params['Settings']['ptypes']:
        if best_param_df == 0:
            best_param_df = param_df[(param_df['Sim'].str.split('-').str[1].eq(p))].sort_values(by=['Score']).head(tuning_params['Settings']['BestN'])
        else:
            best_param_df = best_param_df.append(param_df[(param_df['Sim'].str.split('-').str[1].eq(p))].sort_values(by=['Score']).head(tuning_params['Settings']['BestN']))
    
    # determine new central values for each parameter
    for param in tuning_params:
        if param == 'Settings':
            continue
        else:
            old_val = tuning_params[param][0] # save old value for comparison
            tuning_params[param][0] = best_param_df[param].mean() # set new central value to mean of best simulations
            #if tuning_params[param][0] == old_val:
            #    random.seed(rseed - 1) # better value found, use previous random seed for optimization momentum
    
    # determine current decay of variation
    decay = math.exp(-iteration/tuning_params['Settings']['DecayRate']*math.log(2))
    
    # generate new parameter values
    queued_params = {}
    for param in tuning_params:
        if param == 'Settings':
            continue
        else:
            var = tuning_params[param][0] * tuning_params['Settings']['BaseVar'] * decay / 100
            val0 = tuning_params[param][0]
            queued_params[param] = np.clip(np.random.uniform(val0 - var, val0 + var), tuning_params[param][1], tuning_params[param][2]) # generate new parameter value within bounds
            
            # basic error checking to ensure parameter is within bounds and correct type (int or float)
            if param in ['N', 'PadPlaneX', 'PadPlaneZ', 'PadSizeX', 'PadSizeZ', 'PadRows', 'PadLayers', 'NumTbs', 'YDivider', 'GasFile', 'PadPlaneFile', 'PadShapeFile', 'TB0', 'TBEntrance', 'PeakingTime']:
                queued_params[param] = int(queued_params[param])
            if queued_params[param] * tuning_params[param][0] <= 0:
                queued_params[param] = val0
                
    
    # create new simulations based on new parameter values
    for p in tuning_params['Settings']['ptypes']:
        param_df.loc[len(param_df)] = param_df[(param_df['Sim'].str.split('-').str[1].eq(p))].sort_values(by=['Score']).head(1).iloc[0]
        param_df.loc[len(param_df)-1, 'Sim'] = 'tuning' + str(iteration) + '-' + p
        param_df.loc[len(param_df)-1, 'Status'] = 0
        param_df.loc[len(param_df)-1, 'Score'] = -1
        for param in queued_params:
            param_df.loc[len(param_df)-1, param] = queued_params[param]

    # write param_df to csv
    param_df.to_csv(automation_dir + 'simInput/parameters.csv', index=False)
    
    # write tuning_params to json
    with open(automation_dir + 'simOutput/tuning_params.json', 'w') as f:
        json.dump(tuning_params, f, indent=4)
    
    # write tuning_log to csv
    tuning_log.to_csv(automation_dir + 'simOutput/tuning_log.csv', index=False)
    
    # reset naming of latest output.h5
    os.rename(automation_dir + 'simOutput/' + new_name, automation_dir + 'simOutput/output.h5')

Analyzing file: tuning0-800p.h5
Analyzing file: tuning0-500a.h5
